In [4]:
# Mounting GCS to colab
# https://stackoverflow.com/questions/51715268/how-to-import-data-from-google-cloud-storage-to-google-colab

from google.colab import auth
auth.authenticate_user()

In [5]:
!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!apt -qq update
!apt -qq install gcsfuse

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2537  100  2537    0     0   103k      0 --:--:-- --:--:-- --:--:--  103k
OK
79 packages can be upgraded. Run 'apt list --upgradable' to see them.
gcsfuse is already the newest version (0.34.1).
0 upgraded, 0 newly installed, 0 to remove and 79 not upgraded.


In [6]:
!mkdir colab_indoor
!gcsfuse indoor-data colab_indoor
# !mkdir colab_indoor/train_4
# !gcsfuse indoor-data/train_4 colab_indoor/train_4
# !mkdir colab_indoor/test_4
# !gcsfuse indoor-data/test_4 colab_indoor/test_4

2021/04/22 03:53:55.906575 Using mount point: /content/colab_indoor
2021/04/22 03:53:55.916061 Opening GCS connection...
2021/04/22 03:53:56.179461 Mounting file system "indoor-data"...
2021/04/22 03:53:56.198688 File system has been successfully mounted.


In [7]:
!ls -la -h ./colab_indoor/train_4_colcut

total 1.2G
-rw-r--r-- 1 root root  43M Apr 14 21:07 5a0546857ecc773753327266_train.csv
-rw-r--r-- 1 root root  46M Apr 14 21:07 5c3c44b80379370013e0fd2b_train.csv
-rw-r--r-- 1 root root 113M Apr 14 21:08 5d27075f03f801723c2e360f_train.csv
-rw-r--r-- 1 root root  43M Apr 14 21:08 5d27096c03f801723c31e5e0_train.csv
-rw-r--r-- 1 root root  50M Apr 14 21:08 5d27097f03f801723c320d97_train.csv
-rw-r--r-- 1 root root  12M Apr 14 21:08 5d27099f03f801723c32511d_train.csv
-rw-r--r-- 1 root root  17M Apr 14 21:08 5d2709a003f801723c3251bf_train.csv
-rw-r--r-- 1 root root  72M Apr 14 21:09 5d2709b303f801723c327472_train.csv
-rw-r--r-- 1 root root  81M Apr 14 21:09 5d2709bb03f801723c32852c_train.csv
-rw-r--r-- 1 root root  48M Apr 14 21:09 5d2709c303f801723c3299ee_train.csv
-rw-r--r-- 1 root root  48M Apr 14 21:09 5d2709d403f801723c32bd39_train.csv
-rw-r--r-- 1 root root  51M Apr 14 21:09 5d2709e003f801723c32d896_train.csv
-rw-r--r-- 1 root root 3.9M Apr 14 21:10 5da138274db8ce0c98bbd3d2_train.csv
-

In [38]:
import random
from random import sample
import numpy as np
import pandas as pd
import gc
import matplotlib.pyplot as plt
from scipy.ndimage.filters import uniform_filter1d
from scipy.interpolate import interp1d
import time
from collections import defaultdict
import seaborn as sns

import scipy.stats as stats
from pathlib import Path
import glob
import pickle
from tqdm import tqdm
import random
import os
import copy
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import multiprocessing
import math

EPOCH = 200 # default at 50
BATCH_SIZE = 32
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
FOLDS = 5

NUM_CORES = multiprocessing.cpu_count()
print(NUM_CORES)

OUTPUT_NAME = "train_8_colcut_DLSTM"

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

set_seed()

2


In [9]:
# train paths and test paths
train_files = sorted(glob.glob("./colab_indoor/train_4_colcut/*"))
test_files = sorted(glob.glob("./colab_indoor/test_4_colcut/*"))

# load submission file
sub_df = pd.read_csv("./colab_indoor/sample_submission.csv", index_col=0)
# sub_df[["site", "file", "timestamp"]] = sub_df["site_path_timestamp"].apply(lambda x: pd.Series(x.split("_")))
display(sub_df.head())

,floor,x,y
site_path_timestamp,,,
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000000009,0,75.0,75.0
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000009017,0,75.0,75.0
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000015326,0,75.0,75.0
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000018763,0,75.0,75.0
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000022328,0,75.0,75.0


In [10]:
# Load train csv and test csv
train_df = pd.read_csv(train_files[10])
test_df = pd.read_csv(test_files[10])
display(train_df.head())
display(test_df.head())

,wifi_ts,wps_diff,x,y,floor,floor_int,file_id,site_id,b1847b7279f12430b70cf177ae9419b2c6563c7e,e78f6a70bb684764884bf5bfc22876354dc4978f,7f92c838def910dc333d6ec1785cc9de7c428514,49baf7f0e5495ee2926baa0e941d265d725dbdd5,9419ecb5700d351a0ca4c6f6dc4f7804c5800ebe,6c598641414c17dfca87a3acc982507ba08902cb,bf8984bf3344f3267b1fb3cb5dc95ddf7e2117d3,24a2a9c3edd47eb479f05d4067368c9e14b8964b,2b3f7594cc8ab2b0413d45d6eea1149a9a7809a7,1cecf9fc6a1ceba0ad7063fdb5816ec43e90e94f,018a066872b67d5b33570ddeb6142c602a22e451,55ee0fec7b7be3e76132b20fca01e8486743a3f6,4f32282342971b17eb8611ac5f145d0b8768f62d,ee2a7ce46e83e2de489c9f78f4e550156d57f484,606fc3f5664630412548bbd099f9c8ad1dc5acb6,40ab9973a268cc79a4a8210e67f9b912723ed8c3,c81b745f26735105787cf9e7b547a7c37a65a851,832743c77af1c7eb12fcd909c438dcf39b6008f3,7b42d018fdfe2ab118d8ec85bcf1d6983750750c,ef99ef623422c284a774e35fc1b1cf3e721dbd03,c1591135c00bca7a723a49df8c9d8de34326fbd3,3956907df9b51fae849a7afced246f127f259504,1359034ef247ed40bf5b3723915b9fc60cc74837,2a5b0895595bcf131a0b5c3e5dc4b9f39e06bd3f,ca7a7877b1dbad8f6ef696b1e88f34dc6f635b0d,396640532a81255d6b75b41e7a184c1393b74ee4,aebc6ae6f9dfe7b548779e39cc9fd03c60a67ecc,70e578e027d0ab0bfeb5959e18f3c8045a32236d,07ccdba1f0a266adf501c867319ac7007e65d290,d7b2ba98b972b65650af520075c9712a3d8994a7,2b844704051b64e47b4800e1d6e932bbdd5356b6,1c390c87d362e628cebc103eb247d2b7aa813542,...,46e35eba345a6366234c028567bbb91c99b872e5,298e1870fda64fe8018ec79e835cf308f81181fa,31b0561e7e9a03156b248da215d7ab7bf8d6d8c8,3ab6773bc6bce7fb154b3d46f6058f0192bfd01b,41d0c04a20a77e8daaba6c585f083ca0ef57affc,a3dfc65c01d171a671cd615236b79c61810de354,06028150b43acb7e11121888ab434cdd5f679f1d,f7853f850bbb98d20bb2b4dd6380c071b5a1cb02,078a75dfda9e932f608fca0837610b97d11d3dd7,fc8fd87104388e513937ac13696b6f72af2add53,422765063ea5b0e5667a240e3b802041ab93b6e3,6cfaf9601b6e2587a321d3b803c20f9eb05fd452,35c0ef8cb2af11a81092979d55c4324b8101332f,764cbb8b20a46c411fe5807df9867b92c0935c08,78209eb1032640dd8f611e310c8619ab2c9cb071,1fc19e52666a95169b365f5ad1352d9369cb045a,14d3489b6de7d970babc8a951bf6ed5bd289dea0,6646a9365ace5164d2ef3dfb39b88a8a924568d3,a7ac17ed71cc594bed820bfbbca833f3f9972592,976e778d614b1e8e9efa8bc7eeaf552b7fe5ccb5,5749b4541786aea857ff1552dfac8cf292044c29,b1f4814d1a8c7885960118015e2c8f47fcdd0f4f,75df7c8888ddbf148407aa76c3d6b72406bc4c27,fc88f6a4f0351f8a2b34d256fcda35579d9e0ead,21f079033442b71f1a870ca0988fa9afcc7dc810,3f8321ce0dd6348fa440eb25acae2be996c3b2e7,9054af7ed2ceb6b130b4fd2a09f36b9b1f611f11,77de2f0f3005949e35e2e2aef7af1ddc5d164695,588f35babb83fab68a9c539567140ba5c17284c7,68962923d8621f6868a762b2344ce1ab8209ecdd,7abbb83f5b877ce5c1c9e835422365cd8bac3b0d,acb4abc93968ad51368a0351fa7fda5401598069,f20fed9f248d4352def744d9a8262b49042ee7f3,437d45cd7c45b040b95abdb483b02d601c761d7d,b083b8279418c48d60a05cff039b0536339a1e4d,fa0d8c168b62f11f34fdf314d81cebdf1ca9ebd0,955eb46cfdee70adab1de9ce909f0cb059fd29d2,48d751207622a3606df95184fd58b32347057af3,5238e869a2d0d2a851ba1f9484c874c57419148e,63781517942f8b63c2189c376cc0cf3db1d27ed1
0,1573789444679,1918,88.03561,109.90122,B1,-1,5dce2cf294e4900006124d2a,5d2709d403f801723c32bd39,-76,-74,-76,-999,-73,-89,-77,-999,-49,-999,-73,-87,-999,-999,-999,-46,-66,-75,-84,-999,-76,-999,-999,-999,-84,-999,-999,-999,-999,-999,-999,-80,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
1,1573789446609,3848,88.03561,109.90122,B1,-1,5dce2cf294e4900006124d2a,5d2709d403f801723c32bd39,-79,-79,-83,-999,-73,-89,-77,-999,-44,-88,-78,-87,-999,-999,-999,-59,-66,-75,-89,-999,-76,-999,-999,-999,-84,-999,-999,-999,-999,-999,-999,-80,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
2,1573789448539,5298,91.62208,108.70687,B1,-1,5dce2cf294e4900006124d2a,5d2709d403f801723c32bd39,-68,-67

,site_path_timestamp,correct_wps_ts,wifi_ts,wps_diff,x,y,floor,floor_int,file_id,site_id,b1847b7279f12430b70cf177ae9419b2c6563c7e,e78f6a70bb684764884bf5bfc22876354dc4978f,7f92c838def910dc333d6ec1785cc9de7c428514,49baf7f0e5495ee2926baa0e941d265d725dbdd5,9419ecb5700d351a0ca4c6f6dc4f7804c5800ebe,6c598641414c17dfca87a3acc982507ba08902cb,bf8984bf3344f3267b1fb3cb5dc95ddf7e2117d3,24a2a9c3edd47eb479f05d4067368c9e14b8964b,2b3f7594cc8ab2b0413d45d6eea1149a9a7809a7,1cecf9fc6a1ceba0ad7063fdb5816ec43e90e94f,018a066872b67d5b33570ddeb6142c602a22e451,55ee0fec7b7be3e76132b20fca01e8486743a3f6,4f32282342971b17eb8611ac5f145d0b8768f62d,ee2a7ce46e83e2de489c9f78f4e550156d57f484,606fc3f5664630412548bbd099f9c8ad1dc5acb6,40ab9973a268cc79a4a8210e67f9b912723ed8c3,c81b745f26735105787cf9e7b547a7c37a65a851,832743c77af1c7eb12fcd909c438dcf39b6008f3,7b42d018fdfe2ab118d8ec85bcf1d6983750750c,ef99ef623422c284a774e35fc1b1cf3e721dbd03,c1591135c00bca7a723a49df8c9d8de34326fbd3,3956907df9b51fae849a7afced246f127f259504,1359034ef247ed40bf5b3723915b9fc60cc74837,2a5b0895595bcf131a0b5c3e5dc4b9f39e06bd3f,ca7a7877b1dbad8f6ef696b1e88f34dc6f635b0d,396640532a81255d6b75b41e7a184c1393b74ee4,aebc6ae6f9dfe7b548779e39cc9fd03c60a67ecc,70e578e027d0ab0bfeb5959e18f3c8045a32236d,07ccdba1f0a266adf501c867319ac7007e65d290,d7b2ba98b972b65650af520075c9712a3d8994a7,...,46e35eba345a6366234c028567bbb91c99b872e5,298e1870fda64fe8018ec79e835cf308f81181fa,31b0561e7e9a03156b248da215d7ab7bf8d6d8c8,3ab6773bc6bce7fb154b3d46f6058f0192bfd01b,41d0c04a20a77e8daaba6c585f083ca0ef57affc,a3dfc65c01d171a671cd615236b79c61810de354,06028150b43acb7e11121888ab434cdd5f679f1d,f7853f850bbb98d20bb2b4dd6380c071b5a1cb02,078a75dfda9e932f608fca0837610b97d11d3dd7,fc8fd87104388e513937ac13696b6f72af2add53,422765063ea5b0e5667a240e3b802041ab93b6e3,6cfaf9601b6e2587a321d3b803c20f9eb05fd452,35c0ef8cb2af11a81092979d55c4324b8101332f,764cbb8b20a46c411fe5807df9867b92c0935c08,78209eb1032640dd8f611e310c8619ab2c9cb071,1fc19e52666a95169b365f5ad1352d9369cb045a,14d3489b6de7d970babc8a951bf6ed5bd289dea0,6646a9365ace5164d2ef3dfb39b88a8a924568d3,a7ac17ed71cc594bed820bfbbca833f3f9972592,976e778d614b1e8e9efa8bc7eeaf552b7fe5ccb5,5749b4541786aea857ff1552dfac8cf292044c29,b1f4814d1a8c7885960118015e2c8f47fcdd0f4f,75df7c8888ddbf148407aa76c3d6b72406bc4c27,fc88f6a4f0351f8a2b34d256fcda35579d9e0ead,21f079033442b71f1a870ca0988fa9afcc7dc810,3f8321ce0dd6348fa440eb25acae2be996c3b2e7,9054af7ed2ceb6b130b4fd2a09f36b9b1f611f11,77de2f0f3005949e35e2e2aef7af1ddc5d164695,588f35babb83fab68a9c539567140ba5c17284c7,68962923d8621f6868a762b2344ce1ab8209ecdd,7abbb83f5b877ce5c1c9e835422365cd8bac3b0d,acb4abc93968ad51368a0351fa7fda5401598069,f20fed9f248d4352def744d9a8262b49042ee7f3,437d45cd7c45b040b95abdb483b02d601c761d7d,b083b8279418c48d60a05cff039b0536339a1e4d,fa0d8c168b62f11f34fdf314d81cebdf1ca9ebd0,955eb46cfdee70adab1de9ce909f0cb059fd29d2,48d751207622a3606df95184fd58b32347057af3,5238e869a2d0d2a851ba1f9484c874c57419148e,63781517942f8b63c2189c376cc0cf3db1d27ed1
0,5d2709d403f801723c32bd39_06882da3694b7160c0f10...,1.573706e+12,1573705854189,5859,-999.0,-999.0,-999.0,-999.0,06882da3694b7160c0f105f5,5d2709d403f801723c32bd39,-999,-999,-999,-87,-999,-78,-999,-91,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,...,-999,-999,-999,-999,-999,-999,-999,-80,-999,-999,-999,-76,-85,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-88,-70,-68,-999,-999,-89,-999,-999,-999,-68,-999
1,5d2709d403f801723c32bd39_06882da3694b7160c0f10...,1.573706e+12,1573705854189,174,-999.0,-999.0,-999.0,-999.0,06882da3694b7160c0f105f5,5d2709d403f801723c32bd39,-999,-999,-999,-87,-999,-78,-999,-91,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,...,-999,-999,-999,-999,-999,-999,-999,-80,-999,-999,-999,-76,-85,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-88,-70,-68,-999,-999,-89,-999,-999,-999,-68,-999
2,5d2709d403f801723c32bd39_0

In [11]:
# Match train and test columns
all_train_cols = list(train_df.columns)
all_test_cols = list(test_df.columns)
print("all train cols: ", len(all_train_cols), "\n", "all test cols: ", len(all_test_cols))

# get all non-overlapping columns
no_overlap_col = list(set(all_train_cols) ^ set(all_test_cols))
no_overlap_col += ["floor", "file_id", "site_id"] # add other columns to exclude
train_cols = [x for x in all_train_cols if x not in no_overlap_col]
test_cols = [x for x in all_test_cols if x not in no_overlap_col]
# test_cols += ["site_path_timestamp"] # test_df needs to keep "site_path_timestamp"

# filter out the df by the columns to leave
train_df = train_df[train_cols]
test_df = test_df[test_cols]

# # Drop some columns not necessary as a feature
# drop_cols = ["wifi_ts", "floor", "file_id", "site_id"]
# for df in [train_df, test_df]:
#     df = df.drop(columns=drop_cols, inplace=True)

# Convert df object columns to integers and then the whole thing to tensors
for df in [train_df, test_df]:
    obj_col = list(df.select_dtypes(include=['object']).columns)
    for col in obj_col:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col].values)

# # Try without using scalers for now. 
# # Apply MinMaxScaler to each IMU columns
# current_cols = list(train_df.columns)
# imu_cols = current_cols[:23]
# exception_columns = ["wps_diff", "x", "y", "floor_int", "rel_diff", "rel_x", "rel_y"]
# imu_cols = [x for x in imu_cols if x not in exception_columns]
# print("imu_cols: ", imu_cols)
# for col in imu_cols:
#     ss_scaler = StandardScaler()
#     train_df[col] = ss_scaler.fit_transform(train_df[col].values.reshape(-1, 1))
#     test_df[col] = ss_scaler.transform(test_df[col].values.reshape(-1, 1))

print(len(train_df.columns))
print(len(test_df.columns))
print(len(train_df))
print(len(test_df))
print("object dtype columns in train", train_df.select_dtypes(include=['object']).columns)
print("object dtype columns in test", test_df.select_dtypes(include=['object']).columns)
display(train_df.head())
display(test_df.head())

all train cols:  999 
 all test cols:  1001
996
996
10027
1223
object dtype columns in train Index([], dtype='object')
object dtype columns in test Index([], dtype='object')


,wifi_ts,wps_diff,x,y,floor_int,b1847b7279f12430b70cf177ae9419b2c6563c7e,e78f6a70bb684764884bf5bfc22876354dc4978f,7f92c838def910dc333d6ec1785cc9de7c428514,49baf7f0e5495ee2926baa0e941d265d725dbdd5,9419ecb5700d351a0ca4c6f6dc4f7804c5800ebe,6c598641414c17dfca87a3acc982507ba08902cb,bf8984bf3344f3267b1fb3cb5dc95ddf7e2117d3,24a2a9c3edd47eb479f05d4067368c9e14b8964b,2b3f7594cc8ab2b0413d45d6eea1149a9a7809a7,1cecf9fc6a1ceba0ad7063fdb5816ec43e90e94f,018a066872b67d5b33570ddeb6142c602a22e451,55ee0fec7b7be3e76132b20fca01e8486743a3f6,4f32282342971b17eb8611ac5f145d0b8768f62d,ee2a7ce46e83e2de489c9f78f4e550156d57f484,606fc3f5664630412548bbd099f9c8ad1dc5acb6,40ab9973a268cc79a4a8210e67f9b912723ed8c3,c81b745f26735105787cf9e7b547a7c37a65a851,832743c77af1c7eb12fcd909c438dcf39b6008f3,7b42d018fdfe2ab118d8ec85bcf1d6983750750c,ef99ef623422c284a774e35fc1b1cf3e721dbd03,c1591135c00bca7a723a49df8c9d8de34326fbd3,3956907df9b51fae849a7afced246f127f259504,1359034ef247ed40bf5b3723915b9fc60cc74837,2a5b0895595bcf131a0b5c3e5dc4b9f39e06bd3f,ca7a7877b1dbad8f6ef696b1e88f34dc6f635b0d,396640532a81255d6b75b41e7a184c1393b74ee4,aebc6ae6f9dfe7b548779e39cc9fd03c60a67ecc,70e578e027d0ab0bfeb5959e18f3c8045a32236d,07ccdba1f0a266adf501c867319ac7007e65d290,d7b2ba98b972b65650af520075c9712a3d8994a7,2b844704051b64e47b4800e1d6e932bbdd5356b6,1c390c87d362e628cebc103eb247d2b7aa813542,6d70c76ca1d2e44f634d98810313c1c91b07ffa3,ec097c076cbcdbbcda43d0eb6372101151a914e0,eb61afa796d2ebdf9ec651593837743dc6c3ef11,...,46e35eba345a6366234c028567bbb91c99b872e5,298e1870fda64fe8018ec79e835cf308f81181fa,31b0561e7e9a03156b248da215d7ab7bf8d6d8c8,3ab6773bc6bce7fb154b3d46f6058f0192bfd01b,41d0c04a20a77e8daaba6c585f083ca0ef57affc,a3dfc65c01d171a671cd615236b79c61810de354,06028150b43acb7e11121888ab434cdd5f679f1d,f7853f850bbb98d20bb2b4dd6380c071b5a1cb02,078a75dfda9e932f608fca0837610b97d11d3dd7,fc8fd87104388e513937ac13696b6f72af2add53,422765063ea5b0e5667a240e3b802041ab93b6e3,6cfaf9601b6e2587a321d3b803c20f9eb05fd452,35c0ef8cb2af11a81092979d55c4324b8101332f,764cbb8b20a46c411fe5807df9867b92c0935c08,78209eb1032640dd8f611e310c8619ab2c9cb071,1fc19e52666a95169b365f5ad1352d9369cb045a,14d3489b6de7d970babc8a951bf6ed5bd289dea0,6646a9365ace5164d2ef3dfb39b88a8a924568d3,a7ac17ed71cc594bed820bfbbca833f3f9972592,976e778d614b1e8e9efa8bc7eeaf552b7fe5ccb5,5749b4541786aea857ff1552dfac8cf292044c29,b1f4814d1a8c7885960118015e2c8f47fcdd0f4f,75df7c8888ddbf148407aa76c3d6b72406bc4c27,fc88f6a4f0351f8a2b34d256fcda35579d9e0ead,21f079033442b71f1a870ca0988fa9afcc7dc810,3f8321ce0dd6348fa440eb25acae2be996c3b2e7,9054af7ed2ceb6b130b4fd2a09f36b9b1f611f11,77de2f0f3005949e35e2e2aef7af1ddc5d164695,588f35babb83fab68a9c539567140ba5c17284c7,68962923d8621f6868a762b2344ce1ab8209ecdd,7abbb83f5b877ce5c1c9e835422365cd8bac3b0d,acb4abc93968ad51368a0351fa7fda5401598069,f20fed9f248d4352def744d9a8262b49042ee7f3,437d45cd7c45b040b95abdb483b02d601c761d7d,b083b8279418c48d60a05cff039b0536339a1e4d,fa0d8c168b62f11f34fdf314d81cebdf1ca9ebd0,955eb46cfdee70adab1de9ce909f0cb059fd29d2,48d751207622a3606df95184fd58b32347057af3,5238e869a2d0d2a851ba1f9484c874c57419148e,63781517942f8b63c2189c376cc0cf3db1d27ed1
0,1573789444679,1918,88.03561,109.90122,-1,-76,-74,-76,-999,-73,-89,-77,-999,-49,-999,-73,-87,-999,-999,-999,-46,-66,-75,-84,-999,-76,-999,-999,-999,-84,-999,-999,-999,-999,-999,-999,-80,-999,-999,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
1,1573789446609,3848,88.03561,109.90122,-1,-79,-79,-83,-999,-73,-89,-77,-999,-44,-88,-78,-87,-999,-999,-999,-59,-66,-75,-89,-999,-76,-999,-999,-999,-84,-999,-999,-999,-999,-999,-999,-80,-999,-999,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
2,1573789448539,5298,91.62208,108.70687,-1,-68,-67,-67,-78,-73,-89,-78,-85,-46

,wifi_ts,wps_diff,x,y,floor_int,b1847b7279f12430b70cf177ae9419b2c6563c7e,e78f6a70bb684764884bf5bfc22876354dc4978f,7f92c838def910dc333d6ec1785cc9de7c428514,49baf7f0e5495ee2926baa0e941d265d725dbdd5,9419ecb5700d351a0ca4c6f6dc4f7804c5800ebe,6c598641414c17dfca87a3acc982507ba08902cb,bf8984bf3344f3267b1fb3cb5dc95ddf7e2117d3,24a2a9c3edd47eb479f05d4067368c9e14b8964b,2b3f7594cc8ab2b0413d45d6eea1149a9a7809a7,1cecf9fc6a1ceba0ad7063fdb5816ec43e90e94f,018a066872b67d5b33570ddeb6142c602a22e451,55ee0fec7b7be3e76132b20fca01e8486743a3f6,4f32282342971b17eb8611ac5f145d0b8768f62d,ee2a7ce46e83e2de489c9f78f4e550156d57f484,606fc3f5664630412548bbd099f9c8ad1dc5acb6,40ab9973a268cc79a4a8210e67f9b912723ed8c3,c81b745f26735105787cf9e7b547a7c37a65a851,832743c77af1c7eb12fcd909c438dcf39b6008f3,7b42d018fdfe2ab118d8ec85bcf1d6983750750c,ef99ef623422c284a774e35fc1b1cf3e721dbd03,c1591135c00bca7a723a49df8c9d8de34326fbd3,3956907df9b51fae849a7afced246f127f259504,1359034ef247ed40bf5b3723915b9fc60cc74837,2a5b0895595bcf131a0b5c3e5dc4b9f39e06bd3f,ca7a7877b1dbad8f6ef696b1e88f34dc6f635b0d,396640532a81255d6b75b41e7a184c1393b74ee4,aebc6ae6f9dfe7b548779e39cc9fd03c60a67ecc,70e578e027d0ab0bfeb5959e18f3c8045a32236d,07ccdba1f0a266adf501c867319ac7007e65d290,d7b2ba98b972b65650af520075c9712a3d8994a7,2b844704051b64e47b4800e1d6e932bbdd5356b6,1c390c87d362e628cebc103eb247d2b7aa813542,6d70c76ca1d2e44f634d98810313c1c91b07ffa3,ec097c076cbcdbbcda43d0eb6372101151a914e0,eb61afa796d2ebdf9ec651593837743dc6c3ef11,...,46e35eba345a6366234c028567bbb91c99b872e5,298e1870fda64fe8018ec79e835cf308f81181fa,31b0561e7e9a03156b248da215d7ab7bf8d6d8c8,3ab6773bc6bce7fb154b3d46f6058f0192bfd01b,41d0c04a20a77e8daaba6c585f083ca0ef57affc,a3dfc65c01d171a671cd615236b79c61810de354,06028150b43acb7e11121888ab434cdd5f679f1d,f7853f850bbb98d20bb2b4dd6380c071b5a1cb02,078a75dfda9e932f608fca0837610b97d11d3dd7,fc8fd87104388e513937ac13696b6f72af2add53,422765063ea5b0e5667a240e3b802041ab93b6e3,6cfaf9601b6e2587a321d3b803c20f9eb05fd452,35c0ef8cb2af11a81092979d55c4324b8101332f,764cbb8b20a46c411fe5807df9867b92c0935c08,78209eb1032640dd8f611e310c8619ab2c9cb071,1fc19e52666a95169b365f5ad1352d9369cb045a,14d3489b6de7d970babc8a951bf6ed5bd289dea0,6646a9365ace5164d2ef3dfb39b88a8a924568d3,a7ac17ed71cc594bed820bfbbca833f3f9972592,976e778d614b1e8e9efa8bc7eeaf552b7fe5ccb5,5749b4541786aea857ff1552dfac8cf292044c29,b1f4814d1a8c7885960118015e2c8f47fcdd0f4f,75df7c8888ddbf148407aa76c3d6b72406bc4c27,fc88f6a4f0351f8a2b34d256fcda35579d9e0ead,21f079033442b71f1a870ca0988fa9afcc7dc810,3f8321ce0dd6348fa440eb25acae2be996c3b2e7,9054af7ed2ceb6b130b4fd2a09f36b9b1f611f11,77de2f0f3005949e35e2e2aef7af1ddc5d164695,588f35babb83fab68a9c539567140ba5c17284c7,68962923d8621f6868a762b2344ce1ab8209ecdd,7abbb83f5b877ce5c1c9e835422365cd8bac3b0d,acb4abc93968ad51368a0351fa7fda5401598069,f20fed9f248d4352def744d9a8262b49042ee7f3,437d45cd7c45b040b95abdb483b02d601c761d7d,b083b8279418c48d60a05cff039b0536339a1e4d,fa0d8c168b62f11f34fdf314d81cebdf1ca9ebd0,955eb46cfdee70adab1de9ce909f0cb059fd29d2,48d751207622a3606df95184fd58b32347057af3,5238e869a2d0d2a851ba1f9484c874c57419148e,63781517942f8b63c2189c376cc0cf3db1d27ed1
0,1573705854189,5859,-999.0,-999.0,-999.0,-999,-999,-999,-87,-999,-78,-999,-91,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,...,-999,-999,-999,-999,-999,-999,-999,-80,-999,-999,-999,-76,-85,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-88,-70,-68,-999,-999,-89,-999,-999,-999,-68,-999
1,1573705854189,174,-999.0,-999.0,-999.0,-999,-999,-999,-87,-999,-78,-999,-91,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,...,-999,-999,-999,-999,-999,-999,-999,-80,-999,-999,-999,-76,-85,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-88,-70,-68,-999,-999,-89,-999,-999,-999,-68,-999
2,1573705856155,67,-999.0,-999.0,-999.0,-999,-999,-999,-87,-999,-80,-9

In [ ]:
# # Use when we need to consider timetamps

# # get timestamp and sort by time
# test_df[["site", "file", "timestamp"]] = test_df["site_path_timestamp"].apply(lambda x: pd.Series(x.split("_")))
# test_df = test_df.drop(columns=["site_path_timestamp", "site", "file"])
# test_df["timestamp"] = test_df["timestamp"].astype(int)
# # display(test_df.head())

# # sort by time
# train_df = train_df.sort_values(by=["file_id", "wifi_ts"])
# test_df = test_df.sort_values(by=["file_id", "timestamp"])
# # display(train_df.head(20))
# # display(test_df.head(20))
# # print(len(test_df.columns))

---
## 1Conv + Transformer Implementation
---

In [12]:
class IndoorDataset(Dataset):
    def __init__(self, data, flag="train"):
        self.flag = flag
        self.data = data

        all_cols = list(data.columns)
        target_cols = ["x", "y", "floor_int"]
        non_target_cols = [col for col in all_cols if col not in target_cols]
        self.features = data[non_target_cols]

        if self.flag == "train":
            self.x = data.loc[:, "x"]
            self.y = data.loc[:, "y"]
            # self.f = data.loc[:, "floor_int"]

    def __len__(self):
        return len(self.features)

    def feat_width(self):
        return self.features.shape[1]

    def __getitem__(self, index):
        features = self.features.iloc[index, :]
        features_out = torch.tensor(features.to_numpy())
        if self.flag == "train":
            x = self.x[index]
            y = self.y[index]
            # f = self.f[index]
            x_out = torch.tensor(x)
            y_out = torch.tensor(y)
            # f_out = torch.tensor(f)
            return features_out, x_out, y_out
        else:
            return features_out

In [13]:
# Create train and test Dataset
train_ds = IndoorDataset(train_df)
test_ds = IndoorDataset(test_df, flag="test")

one_train_ds = train_ds.__getitem__(1000)
print("train ds len: ", train_ds.__len__())
# print("train ds features: ", one_train_ds[0])
print("train ds x: ", one_train_ds[1])
print("train ds y: ", one_train_ds[2])
# print("train ds f: ", one_train_ds[3])

one_test_ds = test_ds.__getitem__(0)
print("test ds len: ", test_ds.__len__())
# print("test ds features: ", one_test_ds[0])
# print("test ds: ", one_test_ds)

print(train_ds.feat_width())
print(test_ds.feat_width())

train ds len:  10027
train ds x:  tensor(104.6150, dtype=torch.float64)
train ds y:  tensor(83.2759, dtype=torch.float64)
test ds len:  1223
993
993


In [14]:
# Create Dataloader
train_dataloader = DataLoader(train_ds, batch_size=BATCH_SIZE)
test_dataloader = DataLoader(test_ds, batch_size=BATCH_SIZE)

In [22]:
class Conv1dMaxPool(nn.Module):
    def __init__(self, in_channels, kernel_size, stride, padding):
        super(Conv1dMaxPool, self).__init__()
        self.conv_11 = nn.Conv1d(in_channels, 16, kernel_size, stride, padding)
        self.conv_12 = nn.Conv1d(16, 16, kernel_size, stride, padding)
        self.max_pool_1 = nn.MaxPool1d(kernel_size=3, stride=2, padding=1)
        self.conv_21 = nn.Conv1d(16, 32, kernel_size, stride, padding)
        self.conv_22 = nn.Conv1d(32, 32, kernel_size, stride, padding)
        self.max_pool_2 = nn.MaxPool1d(kernel_size=3, stride=2, padding=1)
        self.conv_31 = nn.Conv1d(32, 64, kernel_size, stride, padding)
        self.conv_32 = nn.Conv1d(64, 64, kernel_size, stride, padding)
        self.max_pool_3 = nn.MaxPool1d(kernel_size=3, stride=2, padding=1)
        self.fc_encoder = nn.Linear(64*2, 512)
        self.batch_norm = nn.BatchNorm1d(512)
        self.out_encoder = nn.Linear(512, 64)

    def forward(self, x, prints=False):
        x = x.unsqueeze(0)
        x = torch.transpose(x, 0, 1)
        if prints: print("before conv 1: ", x.shape)
        x = F.relu(self.conv_11(x))
        x = F.relu(self.conv_12(x))
        x = self.max_pool_1(x)
        if prints: print("after conv & max_pool 1: ", x.shape)

        x = F.relu(self.conv_21(x))
        x = F.relu(self.conv_22(x))
        x = self.max_pool_2(x)
        if prints: print("after conv & max_pool 2: ", x.shape)

        x = F.relu(self.conv_31(x))
        x = F.relu(self.conv_32(x))
        x = self.max_pool_3(x)
        if prints: print("after conv & max_pool 3: ", x.shape)

        # if prints: print("checking reshaping: ", x[0])
        x = x.view(x.size(0), -1) # flatten last two dimensions
        # if prints: print("checking reshaping: ", x[0])
        if prints: print("flatten last two dims: ", x.shape)
        x = self.batch_norm(self.fc_encoder(x))
        x = self.out_encoder(x)
        if prints: print("final output: ", x.shape)
        return x

In [23]:
train_length = train_ds.__len__()
train_width = train_ds.feat_width()

model = Conv1dMaxPool(in_channels=1, kernel_size=3, stride=2, padding=1).to(DEVICE)
print(model)

# input_size = num of features, so length of columns
# sequence_length = 5

# Check if it works
train_batch_sample = next(iter(train_dataloader))
print("feature shape: ", train_batch_sample[0].shape)
print("x shape: ", train_batch_sample[1].shape)
print("y shape: ", train_batch_sample[2].shape)
train_batch_sample = train_batch_sample[0]
print("input shape: ", train_batch_sample.shape)
outputs = model(train_batch_sample.float(), prints=True)
print("final output shape: ", outputs.shape)
print("final output: ", outputs)

Conv1dMaxPool(
  (conv_11): Conv1d(1, 16, kernel_size=(3,), stride=(2,), padding=(1,))
  (conv_12): Conv1d(16, 16, kernel_size=(3,), stride=(2,), padding=(1,))
  (max_pool_1): MaxPool1d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (conv_21): Conv1d(16, 32, kernel_size=(3,), stride=(2,), padding=(1,))
  (conv_22): Conv1d(32, 32, kernel_size=(3,), stride=(2,), padding=(1,))
  (max_pool_2): MaxPool1d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (conv_31): Conv1d(32, 64, kernel_size=(3,), stride=(2,), padding=(1,))
  (conv_32): Conv1d(64, 64, kernel_size=(3,), stride=(2,), padding=(1,))
  (max_pool_3): MaxPool1d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (fc_encoder): Linear(in_features=128, out_features=512, bias=True)
  (batch_norm): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (out_encoder): Linear(in_features=512, out_features=64, bias=True)
)
feature shape:  torch.Size([32, 993])


In [53]:
# Transformer: -> conv1d output -> pe -> (embed thru TransformerEncoderLayer)
# -> encoder -> decoder

class PositionalEncoding(nn.Module):
    def __init__(self, feature_dim, batch_len):
        super(PositionalEncoding, self).__init__()       
        pe = torch.zeros(batch_len, feature_dim)
        position = torch.arange(0, batch_len, dtype=torch.float)
        div_term = torch.exp(torch.arange(0, feature_dim, 2).float() * (-math.log(10000.0) / feature_dim))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        # pe.requires_grad = False
        self.register_buffer('pe', pe)

    def forward(self, x):
        return x + self.pe[:x.size(0), :]

class Transformer(nn.Module):
    def __init__(self, feature_dim, batch_len, num_layers=1, dropout=0.1):
        super(Transformer, self).__init__()
        self.src_mask = None
        self.pos_encoder = PositionalEncoding(feature_dim, batch_len)
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=feature_dim, nhead=2, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)        
        self.decoder = nn.Linear(feature_dim, 2)
        self.init_weights()

    def init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.kaiming_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

    def forward(self, x):
        if self.src_mask is None or self.src_mask.size(0) != len(x):
            mask = self._generate_square_subsequent_mask(len(x)).to(DEVICE)
            self.src_mask = mask

        x = self.pos_encoder(x)
        output = self.transformer_encoder(x, self.src_mask)#, self.src_mask)
        output = self.decoder(output)
        return output

    def _generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

In [55]:
# testing transformer block
Conv = Conv1dMaxPool(in_channels=1, kernel_size=3, stride=2, padding=1).to(DEVICE)

# Check if it works
train_batch_sample = next(iter(train_dataloader))
train_batch_sample = train_batch_sample[0]
conv_outputs = Conv(train_batch_sample.float(), prints=True)
print("final output shape: ", conv_outputs.shape)
print("final output: ", conv_outputs[0])
print(conv_outputs.shape[1])

# check pe
batch_len = conv_outputs.shape[0]
feature_dim = conv_outputs.shape[1]
pe = PositionalEncoding(feature_dim, batch_len)
pe_out = pe(conv_outputs)
print(pe_out[0])
print(pe_out.shape)

# check transformer

before conv 1:  torch.Size([32, 1, 993])
after conv & max_pool 1:  torch.Size([32, 16, 125])
after conv & max_pool 2:  torch.Size([32, 32, 16])
after conv & max_pool 3:  torch.Size([32, 64, 2])
flatten last two dims:  torch.Size([32, 128])
final output:  torch.Size([32, 64])
final output shape:  torch.Size([32, 64])
final output:  tensor([-6.2446e-04, -8.5820e-02,  4.1067e-03, -2.8668e-01,  4.3354e-01,
         1.2378e-01, -8.0052e-02, -5.4131e-01, -7.2685e-01, -5.0350e-01,
        -3.9273e-02, -4.2669e-01,  1.6611e-01,  4.9734e-01, -4.8735e-02,
        -2.8986e-01, -1.8850e-01, -7.3890e-01,  2.4621e-01,  1.5182e-01,
         5.0645e-01,  2.8820e-01,  2.6753e-01, -3.2872e-01, -2.4092e-01,
         1.5652e-01,  6.0274e-01,  3.5113e-02,  5.3187e-01,  4.9030e-01,
         2.4801e-02, -3.1241e-01,  1.0545e-01, -4.1399e-01,  4.4360e-01,
         2.9593e-01, -1.1661e-01, -3.7843e-02, -7.0217e-01, -2.8671e-01,
        -5.2176e-01,  1.3424e-01,  2.0693e-01,  1.7290e-02,  2.0763e-01,
        -2

In [18]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) # default lr: 5e-3

data_dict ={}
best_loss = 1000
best_epoch = 0
model.train()

for epoch in range(EPOCH):
    print("epoch: ", epoch)
    losses = []
    for i, data in enumerate(train_dataloader):
        features = data[0].to(DEVICE).float()
        features = features.unsqueeze(0)
        # print("imu shape: ", imu_features.shape)
        x = data[1].to(DEVICE).float().unsqueeze(-1)
        # print("x shape: ", x.shape)
        y = data[2].to(DEVICE).float().unsqueeze(-1)
        # print("y shape: ", y.shape)

        output = model(features, prints=False)
        # print("output", output)
        # print("output shape after concat: ", output.shape)
        label = torch.cat([x, y], dim=-1)
        # print("label", label)
        # print("label shape after concat: ", label.shape)
        loss = criterion(output, label)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        losses.append(loss.item())
        print(f"loss:{np.mean(losses)} at iteration {i}")

print("Training finished")

epoch:  0
loss:14748.9716796875 at iteration 0
loss:19016.71044921875 at iteration 1
loss:20250.814127604168 at iteration 2
loss:23082.885009765625 at iteration 3
loss:21894.2283203125 at iteration 4
loss:21164.056477864582 at iteration 5
loss:23928.5478515625 at iteration 6
loss:25268.248901367188 at iteration 7
loss:24184.995659722223 at iteration 8
loss:23305.26318359375 at iteration 9
loss:23391.090287642044 at iteration 10
loss:23031.801025390625 at iteration 11
loss:22651.561072716348 at iteration 12
loss:22653.868094308036 at iteration 13
loss:22370.650455729166 at iteration 14


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([32, 2])) that is different to the input size (torch.Size([1, 32, 2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


loss:22248.645446777344 at iteration 15
loss:22196.848517922794 at iteration 16
loss:22257.37060546875 at iteration 17
loss:22668.483604029607 at iteration 18
loss:22784.686181640624 at iteration 19
loss:22536.874116443454 at iteration 20
loss:22330.594859730114 at iteration 21
loss:22617.75972316576 at iteration 22
loss:22760.234415690105 at iteration 23
loss:22393.765 at iteration 24
loss:22455.152869591348 at iteration 25
loss:22502.03146701389 at iteration 26
loss:22327.60518973214 at iteration 27
loss:22379.23114224138 at iteration 28
loss:22215.886783854166 at iteration 29
loss:22412.95425907258 at iteration 30
loss:22676.427612304688 at iteration 31
loss:22810.536280776516 at iteration 32
loss:22826.026137408087 at iteration 33
loss:22828.847544642857 at iteration 34
loss:22928.60910373264 at iteration 35
loss:22767.72994087838 at iteration 36
loss:22947.195929276317 at iteration 37
loss:23043.05659054487 at iteration 38
loss:23304.0626953125 at iteration 39
loss:23413.725752667

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([16, 2])) that is different to the input size (torch.Size([1, 16, 2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


loss:16430.215006510418 at iteration 11
loss:16104.50908954327 at iteration 12
loss:16015.648995535714 at iteration 13
loss:15739.420768229167 at iteration 14
loss:15627.373107910156 at iteration 15
loss:15531.55681295956 at iteration 16
loss:15570.759494357639 at iteration 17
loss:15897.887078536185 at iteration 18
loss:15921.6265625 at iteration 19
loss:15718.552641369048 at iteration 20
loss:15510.034224076704 at iteration 21
loss:15682.983823029892 at iteration 22
loss:15773.520222981771 at iteration 23
loss:15441.43015625 at iteration 24
loss:15447.769981971154 at iteration 25
loss:15428.52586082176 at iteration 26
loss:15250.432547433036 at iteration 27
loss:15259.931640625 at iteration 28
loss:15107.534733072916 at iteration 29
loss:15220.972876764114 at iteration 30
loss:15384.814178466797 at iteration 31
loss:15442.241447679924 at iteration 32
loss:15418.829532398897 at iteration 33
loss:15376.7892578125 at iteration 34
loss:15427.69251844618 at iteration 35
loss:15278.8652871

KeyboardInterrupt: ignored

In [ ]:
# def evaluate(model, data_loader,  device='cuda'):
#     model.to(device)
#     model.eval()
#     x_list = []
#     y_list = []
#     floor_list = []
#     prexs_list = []
#     preys_list = []
#     prefloors_list = []
#     for d in tqdm(data_loader):
#         data_dict['BSSID_FEATS'] = d['BSSID_FEATS'].to(device).long()
#         data_dict['RSSI_FEATS'] = d['RSSI_FEATS'].to(device).float()
#         data_dict['site_id'] = d['site_id'].to(device).long()
#         x = d['x'].to(device).float()
#         y = d['y'].to(device).float()
#         floor = d['floor'].to(device).long()
#         x_list.append(x.cpu().detach().numpy())
#         y_list.append(y.cpu().detach().numpy())
#         floor_list.append(floor.cpu().detach().numpy())
#         xy, floor = model(data_dict)
#         prexs_list.append(xy[:, 0].cpu().detach().numpy())
#         preys_list.append(xy[:, 1].cpu().detach().numpy())
#         prefloors_list.append(floor.squeeze().cpu().detach().numpy())
#     x = np.concatenate(x_list)
#     y = np.concatenate(y_list)
#     floor = np.concatenate(floor_list)
#     prexs = np.concatenate(prexs_list)
#     preys =np.concatenate(preys_list)
#     prefloors = np.concatenate(prefloors_list)
#     eval_score = comp_metric(x, y, floor, prexs, preys, prefloors)
#     return eval_score

# def get_result(model, data_loader, device='cuda'):
#     model.eval()
#     model.to(device)
#     prexs_list = []
#     preys_list = []
#     prefloors_list = []
#     data_dict = {}
#     for d in tqdm(data_loader):
#         data_dict['BSSID_FEATS'] = d['BSSID_FEATS'].to(device).long()
#         data_dict['RSSI_FEATS'] = d['RSSI_FEATS'].to(device).float()
#         data_dict['site_id'] = d['site_id'].to(device).long()
#         xy, floor = model(data_dict)
#         prexs_list.append(xy[:, 0].cpu().detach().numpy())
#         preys_list.append(xy[:, 1].cpu().detach().numpy())
#         prefloors_list.append(floor.squeeze(-1).cpu().detach().numpy())
#     prexs = np.concatenate(prexs_list)
#     preys =np.concatenate(preys_list)
#     prefloors = np.concatenate(prefloors_list)
#     return prexs, preys, prefloors